# <font color=blue> Part I. ETL Pipeline for Pre-Processing the Files</font>

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# <font color=blue> Part II. Populate NoSQL database Apache Cassandra with the information gathered in Part I and validate the database model answering three queries given by the analytics team from Sparkify.  </font>

## The <font color=red>event_datafile_new.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating an Apache Cassandra Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Apache Cassandra Keyspace to handle Sparkify song listen events

In [6]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify_song_listen_events
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set the Keyspace used in this Notebook to be the one created above

In [7]:
# Set KEYSPACE to work on as the above created keyspace 
try:
    session.set_keyspace('sparkify_song_listen_events')
except Exception as e:
    print(e)

#### Set Apache Cassandra row factory as that every SELECT statement returns inside a Pandas DataFrame

In [8]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

try:
    session.row_factory = pandas_factory
except Exception as e:
    print(e)

#PS: special thanks to Giovanni Perna (https://gist.github.com/gioper86/b08b72d77c4e0aefa0137fc3655488dd)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

***

### <font color=green>Answering QUERY 1 :  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4. </font><br>
A column family with its primary key made of **session_id** (as partition key) and **item_in_session** (clustering column) would categorize all the data based on the where filter of query 1 and therefore must meet the expectation.<br>
Trying to translate the query 1 sentence into CQL we have:
```SELECT artist, song, length FROM session_library WHERE session_id = 338 AND item_in_session = 4```
<br>
Below table describes what partition we need to answer query 1

<img src="images/query1.jpg">

In [9]:
create_query = "CREATE TABLE IF NOT EXISTS "
query = create_query + "session_library (\
                              session_id int, \
                              item_in_session int, \
                              artist text, \
                              song text, \
                              length decimal, \
                              PRIMARY KEY (session_id, item_in_session)\
                        )"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Populating session_library table given the information provided on below file 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement based on event_datafile_new.csv organization.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### <font color=red>Executing QUERY 1 to validate the answer</font>

In [22]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM session_library WHERE session_id=338 AND item_in_session=4"
try:
    result_query1 = session.execute(query)
except Exception as e:
    print(e)

# Use a Pandas DataFrame to print the output in a tabular format
df = result_query1._current_rows
df.style.hide_index()

artist,song,length
Faithless,Music Matters (Mark Knight Dub),495.3073


***

### <font color=green>Answering QUERY 2 : Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 </font><br>
A column family with its primary key made of the combination **user_id** and **session_id** as the partition key and **item_in_session** as a clustering column, will have uniqueness based on query 2 where clause while the partition data will be ordered by **item_in_session** <br>
Trying to translate the query 2 sentence into CQL we have:
```SELECT artist, song, user_first_name, user_last_name FROM user_library WHERE user_id = 10 AND session_id = 182```
Below table describes what partition we need to answer query 2<br>

<img src="images/query2.jpg">

In [12]:
create_query = "CREATE TABLE IF NOT EXISTS "
query = create_query + "user_library (\
                              user_id int, \
                              session_id int, \
                              item_in_session int, \
                              artist text, \
                              song text, \
                              user_first_name text, \
                              user_last_name text, \
                              PRIMARY KEY ((user_id, session_id), item_in_session) \
                        )"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
# Populating user_library table given the information provided on below file 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (user_id, session_id, item_in_session, artist, song, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement based on event_datafile_new.csv organization.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### <font color=red>Executing QUERY 2 to validate the answer</font>

In [21]:
query = "SELECT artist, song, user_first_name, user_last_name FROM user_library WHERE user_id = 10 AND session_id = 182"
try:
    result_query2 = session.execute(query)
except Exception as e:
    print(e)

# Use a Pandas DataFrame to print the output in a tabular format
df = result_query2._current_rows
df['user_name'] = df.user_first_name + " " + df.user_last_name
df.drop(['user_first_name', 'user_last_name'], axis=1, inplace=True)
df.style.hide_index()

artist,song,user_name
Down To The Bone,Keep On Keepin' On,Sylvie Cruz
Three Drives,Greece 2000,Sylvie Cruz
Sebastien Tellier,Kilometer,Sylvie Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie Cruz


***

### <font color=green>Answering query 3 : Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'</font><br>
The key to this query is to understand that we are interested in the **"listened to"** event to guarantee uniqueness, not on the music by itself. <br>
The **listened to** event is made up of the combination of **session_id**, **item_in_session** and **song** and therefore we can make it the primary key of our partition.<br>
But, as query 3 filter is based upon the song, the partition key, i.e. the first element in the primary key, must be the **song** otherwise Apache Cassandra wouldn't be able to filter **song** column without filtering its preceding column before. <br>
Trying to translate the query 3 sentence into CQL we have:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
```SELECT user_first_name, user_last_name FROM listened_to_library WHERE song='All Hands Against His Own'```
Below table describes what partition we need to answer query 3
<br>

<img src="images/query3.jpg">
                    

In [23]:
create_query = "CREATE TABLE IF NOT EXISTS "
query = create_query + "listened_to_library (\
                              session_id int, \
                              item_in_session int, \
                              song text, \
                              user_first_name text, \
                              user_last_name text, \
                              PRIMARY KEY (song, session_id, item_in_session) \
                        )"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [24]:
# Populating listened_to_library table given the information provided on below file 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO listened_to_library (session_id, item_in_session, song, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement based on event_datafile_new.csv organization.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[9], line[1], line[4]))

#### <font color=red>Executing QUERY 3 to validate the answer</font>

In [ ]:
query = "SELECT user_first_name, user_last_name FROM listened_to_library WHERE song='All Hands Against His Own'"
try:
    result_query2 = session.execute(query)
except Exception as e:
    print(e)

# Use a Pandas DataFrame to print the output in a tabular format
df = result_query2._current_rows
df['user_name'] = df.user_first_name + " " + df.user_last_name
df.drop(['user_first_name', 'user_last_name'], axis=1, inplace=True)
df.style.hide_index()

for row in result_query2:
    print("\033[1m User: \033[0m"+ row[0] + " " + row[1])

***

### Drop the tables before closing out the sessions

In [ ]:
## Drop the tables before closing out the sessions
try:
    session.execute("""DROP TABLE IF EXISTS session_library""")
except Exception as e:
    print(e)
try:
    session.execute("""DROP TABLE IF EXISTS user_library""")
except Exception as e:
    print(e)
try:
    session.execute("""DROP TABLE IF EXISTS listened_to_library""")
except Exception as e:
    print(e)

In [ ]:
# Drop the KEYSPACE before closing out the sessions
try:
    session.execute("""DROP KEYSPACE IF EXISTS sparkify_song_listen_events""")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()